In [ ]:
import AsyncDataSetQuery

environmentName = 'JALOCAL'

query = AsyncDataSetQuery.AsyncDataSetQuery( 'ws://localhost:9000',environmentName)

path = 'C:\\Earthwave\\MountainGlaciers\\files'
startsWith = 'CS'
endsWith = '.nc'
columns = ['coh','power']

#data = query.validateNetCdf( path, startsWith, endsWith, columns )



In [ ]:
import datetime

envName = 'JALOCAL'
parentDs = 'test'
dataSetName ='ReadyData'
region ='himalayas'
minX=0
maxX=100000
minY=100000
maxY=200000
minT=datetime.datetime(2010,12,1,0,0)
maxT=datetime.datetime(2010,12,31,0,0)

projections = []
filters = []

filename = query.executeQuery( parentDs, dataSetName,region, minX, maxX, minY, maxY, minT, maxT, projections, filters )

print(filename.resultFileName)
print(filename.status)
print(filename.json)





In [ ]:
envName = 'JALOCAL'
parentDs = 'test'
dataSetName ='ReadyData'
region ='himalayas'
projection = 'HMA'
sourceFileName = 'C:\\Earthwave\\mtngla\\tandemx\\swath\\y2010\\m12\\cell_HMA_0_100000\\GridCell_0.nc'
minX = 0
minY = 100000
size = 100000


results = query.publishGridCellPoints( parentDs, dataSetName, region, minX, minY, size, sourceFileName, projection)

In [ ]:
print(results)